In [28]:
# Mateusz Chudzik
# WCY21KS1S4 

from random import randint
from time import time

def Cantor_half_reduction(D_1, D_2, f, h):

    a1 = D_1[0]
    a2 = D_2[0]
    
    b1 = D_1[1]
    b2 = D_2[1]
    
    d1, e1, e2 = xgcd(a1, a2)

    d, c1, c2 = xgcd(d1, b1+b2+h)

    s1 = c1*e1
    s2 = c1*e2 
    s3 = c2
    
    a = R((a1*a2) / (d**2))

    b = (R((s1*a1*b2) + (s2*a2*b1) + (s3*(b1*b2 + f))) * d.inverse_mod(a)).mod(a)

    return R(a), R(b)

def Cantor_reduction(D, f, g):
    a = D[0]
    b = D[1]
    
    h = 0
    
    a_prim = R(R(f - (b*h) - (b**2)) / R(a))
    b_prim = R((-h - b)).mod(a_prim)

    deg_a_prim = R(a_prim).degree()

    while deg_a_prim > g:

        a_prim = R(R(f - (b*h) - (b^2)) / R(a))
        b_prim = R((-h - b)).mod(a_prim)
        
        a = a_prim
        b = b_prim

        deg_a_prim = R(a_prim).degree()

    return R(a_prim / a_prim.coefficients()[-1]), R(b_prim)
    
def Montgomery_ladder(D, k, f, g, h):
    R0 = D
    R1 = Cantor_half_reduction(D, D, f, h)
    a, b = R1
    if a.degree() > g:
        R1 = Cantor_reduction(R1, f, g)
    K = bin(k)[3:]
    for i in range(0, len(K)):
        if K[i] == '1':
            R0 = Cantor_half_reduction(R1, R0, f, h)
            a, b = R0
            if a.degree() > g:
                R0 = Cantor_reduction(R0, f, g)
            
            R1 = Cantor_half_reduction(R1, R1, f, h)
            a, b = R1
            if a.degree() > g:
                R1 = Cantor_reduction(R1, f, g)
        else:
            R1 = Cantor_half_reduction(R0, R1, f, h)
            a, b = R1
            if a.degree() > g:
                R1 = Cantor_reduction(R1, f, g)
                
            R0 = Cantor_half_reduction(R0, R0, f, h)
            a, b = R0
            if a.degree() > g:
                R0 = Cantor_reduction(R0, f, g)
    return R0

def divisor_add(D_1, D_2, f, g, h):
    D = Cantor_half_reduction(D_1, D_2, f, h)
    a, b = D
    if a.degree() > g:
        D = Cantor_reduction(D, f, g)
    return D


def BSGS(D, alfa_D, a, b, f, g, h):
    
    alfa_D_inv = (alfa_D[0], - alfa_D[1] - h)
    k = ceil(sqrt(abs(b - a)))
    
    a_D = Montgomery_ladder(D, a, f, g, h)
    k_D = Montgomery_ladder(D, k, f, g, h)
    K = {(1, 0) : 0, (D[0], D[1]) : 1}
    
    for i in range(2, k):
        K[Montgomery_ladder(D, i, f, g, h)] = i
    
    temp = (1, 0)
    
    alfa = 0
    
    for l1 in range(k):
        temp = divisor_add(a_D, alfa_D_inv, f, g, h)        
        l1_k_D = Montgomery_ladder(k_D, l1, f, g, h)

        l2_D = divisor_add(temp, l1_k_D, f, g, h)
        try:
            l2 = K[l2_D]
            alfa = a + (l1 * k) - l2
            break
        except:
            next
    return alfa
      
def unit_test_BSGS(D_list, D, AB, f, g, h, p, C, ord):

    for i, alfa_D in enumerate(D_list):
        a, b = AB[i]
        t1 = time()
        alfa = BSGS(D, alfa_D, a, b, f, g, h)
        t2 = time()
    
        refference_alfa_D = Montgomery_ladder(D, alfa, f, g, h)
        if alfa_D == refference_alfa_D:
            print("=" * 100)
            print("Poprawnie obliczono logarytm dyskretny dla przypadku:")
            print("Cialo: F_"+str(p))
            print("f(x) = " + str(f))
            print("Krzywa hipereliptyczna: "+str(C))
            print("Rzad krzywej: "+ str(ord))
            print("[alfa]D = " + str(alfa_D))
            print("D = " + str(D))
            print("alfa = " + str(alfa))
            print("a, b = "+ str((a, b)))
            print("alfa została policzona w : " + str(t2 - t1) + " sekund")
            print("Dlugosc bitowa charakterystyki ciala: " + str(len(bin(p)[2:])) + "\n")
            
    
p = 4099
F = GF(p)
R.<x> = F[]
f = x^5 + 2745*x^4 + 3512*x^3 + 3653*x^2 + 2633*x + 695
C =  HyperellipticCurve(f)                                                 
J = C.jacobian()                                                          
JJ = J(F)
ord = 16875613
P = C([1348 , 3520])
D = JJ(P)
g = C.genus()
h = 0

D1 = (x^2 + 3590*x + 2384, 3966*x + 1028) 
D2 = (x^2 + 619*x + 3104, 3019*x + 3540) 
D3 = (x^2 + 211*x + 1856, 3121*x + 3481) 
D4 = (x^2 + 2699*x + 2815, 3352*x + 1835) 

D_list_1 = [D1, D2, D3, D4]
AB = []


a =  6750144
b =  6750344
AB += [(a, b)]

#D1 = [alfa]D : a(x) =  x^2 + 3590*x + 2384  b(x) =  3966*x + 1028


a =  1686561
b =  1688561
AB += [(a, b)]

#D2 = [alfa]D : a(x) =  x^2 + 619*x + 3104  b(x) =  3019*x + 3540

a =  6740244
b =  6760244
AB += [(a, b)]

#D3 = [alfa]D : a(x) =  x^2 + 211*x + 1856  b(x) =  3121*x + 3481

a =  2549794
b =  10950694
AB += [(a, b)]

#D4 = [alfa]D : a(x) =  x^2 + 2699*x + 2815  b(x) =  3352*x + 1835

unit_test_BSGS(D_list_1, D, AB, f, g, h, p, C, ord)

p = 1048583
F = GF(p)
R.<x> = F[]
f = x^5 + 812044*x^4 + 543246*x^3 + 230741*x^2 + 825190*x + 633221
C =  HyperellipticCurve(f)                                                 
J = C.jacobian()                                                          
JJ =J(F)
ord = 1099305873793
P = C([563624, 291890])
D = JJ(P)
g = C.genus()
h = 0


D1 = (x^2 + 1035404*x + 618950, 1009268*x + 301386)
D2 = (x^2 + 227176*x + 987279, 421850*x + 537957)
D3 = (x^2 + 436913*x + 398789, 455865*x + 827852)

D_list_2 = [D1, D2, D3]
AB = []

# 1. 
a =  329791761637
b =  329791762637
AB += [(a, b)]

# D1 = [alfa]D : a(x) =  x^2 + 1035404*x + 618950  b(x) =  1009268*x + 301386

# 2.
a =  439722344516
b =  439722354516
AB += [(a, b)]

# D2 = [alfa]D : a(x) =  x^2 + 227176*x + 987279  b(x) =  421850*x + 537957

# 3.
a =  549652886897
b =  549652986897
AB += [(a, b)]

# D3 = [alfa]D : a(x) =  x^2 + 436913*x + 398789  b(x) =  455865*x + 827852

unit_test_BSGS(D_list_2, D, AB, f, g, h, p, C, ord)

Poprawnie obliczono logarytm dyskretny dla przypadku:
Cialo: F_4099
f(x) = x^5 + 2745*x^4 + 3512*x^3 + 3653*x^2 + 2633*x + 695
Krzywa hipereliptyczna: Hyperelliptic Curve over Finite Field of size 4099 defined by y^2 = x^5 + 2745*x^4 + 3512*x^3 + 3653*x^2 + 2633*x + 695
Rzad krzywej: 16875613
[alfa]D = (x^2 + 3590*x + 2384, 3966*x + 1028)
D = (x + 2751, y + 579)
alfa = 6750211
a, b = (6750144, 6750344)
alfa została policzona w : 0.011924505233764648 sekund
Dlugosc bitowa charakterystyki ciala: 13

Poprawnie obliczono logarytm dyskretny dla przypadku:
Cialo: F_4099
f(x) = x^5 + 2745*x^4 + 3512*x^3 + 3653*x^2 + 2633*x + 695
Krzywa hipereliptyczna: Hyperelliptic Curve over Finite Field of size 4099 defined by y^2 = x^5 + 2745*x^4 + 3512*x^3 + 3653*x^2 + 2633*x + 695
Rzad krzywej: 16875613
[alfa]D = (x^2 + 619*x + 3104, 3019*x + 3540)
D = (x + 2751, y + 579)
alfa = 1687738
a, b = (1686561, 1688561)
alfa została policzona w : 0.0434114933013916 sekund
Dlugosc bitowa charakterystyki ciala: 1

In [20]:
Montgomery_ladder(D, 439722350282, f, g, h)

(x^2 + 227176*x + 987279, 421850*x + 537957)